In [106]:
%load_ext autoreload
%autoreload 2

import sympy as sp
from sympy import Derivative, Symbol, IndexedBase, Idx, Sum, Function, Matrix, Add, Eq
from sympy import diff, symbols, solve, linsolve, simplify, poly, pprint, factor, integrate, lambdify
from sympy import cos, sin, pi

# syspath trick
import sys
sys.path.append('../')

from tatarinov.database.singleton import db
db.set_root('../data/triangular_platform')


from tatarinov.utils.jupyter import display_list, display_obj
from tatarinov.core.variables import _omega, _v
from tatarinov.core.tatarinov import TatarinovSystem

from tatarinov.problems.triangular_platform.variables import *
from tatarinov.problems.triangular_platform.structure import *
from tatarinov.problems.triangular_platform.functions import euler, scalar
from tatarinov.utils.sympy import MatrixElDerivative

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Посмотрим, будут ли не зависеть от времени момент при движении по окружности

In [107]:
TWNU3 = db.load('TWNU3').obj
constraints = db.load('constraints').obj
nu1, nu2 = db.load('nu1,nu2(x,y)').obj

Loading from: ../data/triangular_platform/TWNU3.pickle
Loading from: ../data/triangular_platform/constraints.pickle
Loading from: ../data/triangular_platform/nu1,nu2(x,y).pickle


In [108]:
display_list([nu1, nu2])

Eq(nu1(t), sin(alpha(t))*Derivative(y(t), t) + cos(alpha(t))*Derivative(x(t), t))

Eq(nu2(t), -sin(alpha(t))*Derivative(x(t), t) + cos(alpha(t))*Derivative(y(t), t))

In [109]:
equations = [TWNU3[2]] + [constraints[2], constraints[3]]

In [110]:
display_list(equations)

Eq(Derivative(nu3(t), t), (-nu1(t)**2*sin(theta0(t))**3 + 3*nu1(t)**2*sin(theta0(t))/4 - nu1(t)**2*sin(3*theta0(t))/4 - 2*nu1(t)*nu2(t)*cos(theta0(t))**3 + 3*nu1(t)*nu2(t)*cos(theta0(t))/2 + nu1(t)*nu2(t)*cos(3*theta0(t))/2 - 4*nu1(t)*nu3(t)*cos(theta0(t))**3 + 3*nu1(t)*nu3(t)*cos(theta0(t))**2 + 9*nu1(t)*nu3(t)*cos(theta0(t)) + nu1(t)*nu3(t)*cos(3*theta0(t)) + nu2(t)**2*sin(theta0(t))**3 - 3*nu2(t)**2*sin(theta0(t))/4 + nu2(t)**2*sin(3*theta0(t))/4 + 4*nu2(t)*nu3(t)*sin(theta0(t))**3 + 3*nu2(t)*nu3(t)*sin(theta0(t))*cos(theta0(t)) - 3*nu2(t)*nu3(t)*sin(theta0(t)) + nu2(t)*nu3(t)*sin(3*theta0(t)) + 4*nu3(t)**2*sin(theta0(t))**3 - 3*nu3(t)**2*sin(theta0(t)) + nu3(t)**2*sin(3*theta0(t)) - 4*sin(theta0(t))*cos(theta0(t))*Derivative(nu1(t), t) + 4*cos(theta0(t))**2*Derivative(nu2(t), t) + 6*cos(theta0(t))*Derivative(nu2(t), t))/(3*cos(theta0(t)) - cos(2*theta0(t)) - 1))

Eq(Derivative(alpha(t), t), nu3(t))

Eq(Derivative(theta0(t), t), (-R*nu3(t)*cos(beta0 - theta0(t)) - d*nu3(t) + nu1(t)*sin(theta0(t)) - nu2(t)*cos(theta0(t)))/d)

In [129]:
const, V, w = symbols('(const), V, omega')

subs_dict = {
    x: V*cos(w*t),
    y: V*sin(w*t),
    beta[0]: 0 # !
}

subs_dict.update({
    nu[1]: nu1.subs(subs_dict).args[1],
    nu[2]: nu2.subs(subs_dict).args[1]
})

subs_dict_const = {
    nu[3]: symbols('C'),
    Derivative(alpha, t): symbols('C')
}

subs_dict.update(subs_dict_const)

In [130]:
%%time 
equations1 = [simplify(_.subs(subs_dict)) for _ in equations[0::2]]

CPU times: user 37.1 s, sys: 7.85 ms, total: 37.1 s
Wall time: 37.1 s


In [131]:
equations1

[Eq((-4*C**2*sin(theta0(t))**3 + 3*C**2*sin(theta0(t)) - C**2*sin(3*theta0(t)) + 4*C*(-V*omega*sin(omega*t)*sin(alpha(t)) - V*omega*cos(omega*t)*cos(alpha(t)))*sin(theta0(t))**3 + 3*C*(-V*omega*sin(omega*t)*sin(alpha(t)) - V*omega*cos(omega*t)*cos(alpha(t)))*sin(theta0(t))*cos(theta0(t)) - 3*C*(-V*omega*sin(omega*t)*sin(alpha(t)) - V*omega*cos(omega*t)*cos(alpha(t)))*sin(theta0(t)) + C*(-V*omega*sin(omega*t)*sin(alpha(t)) - V*omega*cos(omega*t)*cos(alpha(t)))*sin(3*theta0(t)) + 4*C*(-V*omega*sin(omega*t)*cos(alpha(t)) + V*omega*sin(alpha(t))*cos(omega*t))*cos(theta0(t))**3 - 3*C*(-V*omega*sin(omega*t)*cos(alpha(t)) + V*omega*sin(alpha(t))*cos(omega*t))*cos(theta0(t))**2 - 9*C*(-V*omega*sin(omega*t)*cos(alpha(t)) + V*omega*sin(alpha(t))*cos(omega*t))*cos(theta0(t)) - C*(-V*omega*sin(omega*t)*cos(alpha(t)) + V*omega*sin(alpha(t))*cos(omega*t))*cos(3*theta0(t)) + 4*V*omega*(omega - Derivative(alpha(t), t))*sin(omega*t - alpha(t))*cos(theta0(t))**2 + 6*V*omega*(omega - Derivative(alpha(t),

### Необходимые уравнения

In [132]:
equations2 = equations1 + [equations[1]]

In [133]:
display_list(equations2)

Eq((-4*C**2*sin(theta0(t))**3 + 3*C**2*sin(theta0(t)) - C**2*sin(3*theta0(t)) + 4*C*(-V*omega*sin(omega*t)*sin(alpha(t)) - V*omega*cos(omega*t)*cos(alpha(t)))*sin(theta0(t))**3 + 3*C*(-V*omega*sin(omega*t)*sin(alpha(t)) - V*omega*cos(omega*t)*cos(alpha(t)))*sin(theta0(t))*cos(theta0(t)) - 3*C*(-V*omega*sin(omega*t)*sin(alpha(t)) - V*omega*cos(omega*t)*cos(alpha(t)))*sin(theta0(t)) + C*(-V*omega*sin(omega*t)*sin(alpha(t)) - V*omega*cos(omega*t)*cos(alpha(t)))*sin(3*theta0(t)) + 4*C*(-V*omega*sin(omega*t)*cos(alpha(t)) + V*omega*sin(alpha(t))*cos(omega*t))*cos(theta0(t))**3 - 3*C*(-V*omega*sin(omega*t)*cos(alpha(t)) + V*omega*sin(alpha(t))*cos(omega*t))*cos(theta0(t))**2 - 9*C*(-V*omega*sin(omega*t)*cos(alpha(t)) + V*omega*sin(alpha(t))*cos(omega*t))*cos(theta0(t)) - C*(-V*omega*sin(omega*t)*cos(alpha(t)) + V*omega*sin(alpha(t))*cos(omega*t))*cos(3*theta0(t)) + 4*V*omega*(omega - Derivative(alpha(t), t))*sin(omega*t - alpha(t))*cos(theta0(t))**2 + 6*V*omega*(omega - Derivative(alpha(t), 

Eq(Derivative(theta0(t), t), (-C*R*cos(theta0(t)) - C*d - V*omega*sin(omega*t)*sin(alpha(t) + theta0(t)) - V*omega*cos(omega*t)*cos(alpha(t) + theta0(t)))/d)

Eq(Derivative(alpha(t), t), nu3(t))

In [134]:
# x(t),y(t) ---> [T[1], W[1], Derivative(alpha, t)]

In [135]:
equations3 = [_.subs(subs_dict_const) for _ in equations2]

In [136]:
display_list(equations3)

Eq((-4*C**2*sin(theta0(t))**3 + 3*C**2*sin(theta0(t)) - C**2*sin(3*theta0(t)) + 4*C*(-V*omega*sin(omega*t)*sin(alpha(t)) - V*omega*cos(omega*t)*cos(alpha(t)))*sin(theta0(t))**3 + 3*C*(-V*omega*sin(omega*t)*sin(alpha(t)) - V*omega*cos(omega*t)*cos(alpha(t)))*sin(theta0(t))*cos(theta0(t)) - 3*C*(-V*omega*sin(omega*t)*sin(alpha(t)) - V*omega*cos(omega*t)*cos(alpha(t)))*sin(theta0(t)) + C*(-V*omega*sin(omega*t)*sin(alpha(t)) - V*omega*cos(omega*t)*cos(alpha(t)))*sin(3*theta0(t)) + 4*C*(-V*omega*sin(omega*t)*cos(alpha(t)) + V*omega*sin(alpha(t))*cos(omega*t))*cos(theta0(t))**3 - 3*C*(-V*omega*sin(omega*t)*cos(alpha(t)) + V*omega*sin(alpha(t))*cos(omega*t))*cos(theta0(t))**2 - 9*C*(-V*omega*sin(omega*t)*cos(alpha(t)) + V*omega*sin(alpha(t))*cos(omega*t))*cos(theta0(t)) - C*(-V*omega*sin(omega*t)*cos(alpha(t)) + V*omega*sin(alpha(t))*cos(omega*t))*cos(3*theta0(t)) + 4*V*omega*(-C + omega)*sin(omega*t - alpha(t))*cos(theta0(t))**2 + 6*V*omega*(-C + omega)*sin(omega*t - alpha(t))*cos(theta0(t))

Eq(Derivative(theta0(t), t), (-C*R*cos(theta0(t)) - C*d - V*omega*sin(omega*t)*sin(alpha(t) + theta0(t)) - V*omega*cos(omega*t)*cos(alpha(t) + theta0(t)))/d)

True

In [143]:
_0 = trigsimp(equations3[1]).doit()
_0

Eq(Derivative(theta0(t), t), -(C*R*cos(theta0(t)) + C*d + V*omega*cos(-omega*t + alpha(t) + theta0(t)))/d)

### Заметим, что при $\alpha = \omega t$:

In [145]:
_1 = _0.subs({
    alpha: w*t,
    Derivative(theta[0], t): 0
}).doit()
_1

Eq(0, -(C*R*cos(theta0(t)) + C*d + V*omega*cos(theta0(t)))/d)

### и так как ищется постоянное решение, то приравняем правую часть нулю и разрешим относительно $\cos{(\beta_{0} - \theta)}$ чтобы потом подставить в уравнение на $\nu_3$

In [149]:
_sol = solve(_1, cos(theta[0]))
_sol

[-C*d/(C*R + V*omega)]

In [150]:
_2 = trigsimp(equations3[0]).doit()
_2

Eq((-4*C**2*sin(theta0(t))**3 + 3*C**2*sin(theta0(t)) - C**2*sin(3*theta0(t)) - 3*C*V*omega*(sin(-omega*t + alpha(t) + 2*theta0(t)) + sin(omega*t - alpha(t) + 2*theta0(t)))/4 - 4*C*V*omega*sin(omega*t - alpha(t))*cos(theta0(t))**3 + 3*C*V*omega*sin(omega*t - alpha(t))*cos(theta0(t))**2 + 9*C*V*omega*sin(omega*t - alpha(t))*cos(theta0(t)) + C*V*omega*sin(omega*t - alpha(t))*cos(3*theta0(t)) - 4*C*V*omega*sin(theta0(t))**3*cos(omega*t - alpha(t)) + 3*C*V*omega*sin(theta0(t))*cos(omega*t - alpha(t)) - C*V*omega*sin(3*theta0(t))*cos(omega*t - alpha(t)) + 3*V**2*omega**2*(-sin(-2*omega*t + 2*alpha(t) + theta0(t)) + sin(2*omega*t - 2*alpha(t) + theta0(t)))/8 + V**2*omega**2*(-sin(-2*omega*t + 2*alpha(t) + 3*theta0(t)) + sin(2*omega*t - 2*alpha(t) + 3*theta0(t)))/8 + V**2*omega**2*sin(omega*t - alpha(t))**2*sin(theta0(t))**3 - 3*V**2*omega**2*sin(omega*t - alpha(t))**2*sin(theta0(t))/4 + V**2*omega**2*sin(omega*t - alpha(t))**2*sin(3*theta0(t))/4 - 2*V**2*omega**2*sin(omega*t - alpha(t))*cos(

In [153]:
_3 = trigsimp(_2.subs({
    alpha: w*t,
    cos(theta[0]): _sol[0],
    symbols('C'): w, # Почему? 
}))
_3

Eq(-3*V*omega**2*sin(2*theta0(t))/(2*(3*d*omega/(R*omega + V*omega) + cos(2*theta0(t)) + 1)), 0)